<a href="https://colab.research.google.com/github/Vini-Sa/chatbot_pln/blob/main/ProjetoFinal_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Projeto final da Disciplina de Processamento de Linguagem Natural: Assistente Virtual do SEI Julgar

Bem-vindo ao notebook de desenvolvimento do **Assistente Virtual do SEI Julgar**!

Este projeto demonstra como construir um chatbot de **Recuperação Aumentada por Geração (RAG)**, especializado em fornecer suporte técnico e instruções detalhadas sobre o uso do módulo **SEI Julgar**.

Projeto desenvolvido por:

André Cacau

Vinícius de Souza Sá

---

**Problemas e Justificativa do Trabalho**

Como estavámos desenvolvendo o manual do SEI Julgar, e ele é bem extenso e detalhado, os usuários teriam relutância em buscar no material a resposta para suas dúvidas. Logo, o chatbot surge para possibilitar um acesso rápido, dinâmico e intuitivo às informações por meio de conversas.

---
**Objetivo do trabalho**

Chatbot com RAG (*Retrieval-Augmented Generation*) que responde com base no manual do SEI Julgar.


## Instalação das bibliotecas

In [ ]:
!pip install -q \
    langchain langchain-groq langchain_community langchain-huggingface \
    faiss-cpu chromadb PyPDF2 streamlit python-dotenv

!npm install -g localtunnel
!python -m spacy download pt_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 28.6 MB/s eta 

## Importe o manual Documentação SEI Julgar - Secretaria.pdf

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Arquivo enviado: {filename}")


Saving Documentação SEI Julgar - Secretaria.pdf to Documentação SEI Julgar - Secretaria.pdf
Arquivo enviado: Documentação SEI Julgar - Secretaria.pdf


## Módulo de Pré-processamento e Análise de Linguagem Natural (PLN)
Para iniciar o desenvolvimento do chatbot, adicionamos uma parte de pré processamento inteligente de PLN antes de enviar a pergunta do usuário ao LLM, para tornar o texto limpo, compreensível e enriquecido semanticamente.

| Etapa | Função Principal | Objetivo |
| :--- | :--- | :--- |
| **1. Normalização** | `normalize_text` | Limpar, padronizar e remover palavras irrelevantes (*stopwords*). |
| **2. Análise de Intenção** | `analyze_intent` | Classificar a necessidade do usuário (Instrução, Justificativa, Identificação). |
| **3. Análise de Sentimento** | `analyze_sentiment` | Detectar o tom emocional (Positivo, Negativo, Neutro). |
| **4. Extração de Entidades** | `extract_entities` | Identificar e catalogar nomes (Pessoas, Lugares, Organizações) no texto. |
| **5. Expansão Semântica** | `semantic_expansion` | Adicionar termos semanticamente relacionados (sinônimos) para aprimorar a busca. |


In [ ]:

import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

# 1. Configuração inicial - Carregar o modelo de linguagem do spaCy em português
try:
    nlp = spacy.load("pt_core_news_md")
except Exception as e:
    try:
        nlp = spacy.load("pt_core_news_sm")
    except Exception as e2:
        st.error("Erro: Modelo spaCy não encontrado. Execute no notebook: !python -m spacy download pt_core_news_md")
        st.stop()

# Configuração de embeddings - converte textos em vetores numéricos
embedder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Configuração de stopwords - palavras que não têm valor semântico
try:
    stop_words = set(stopwords.words('portuguese'))
except Exception:
    stop_words = set()

# 1. Normalização
# Remove caracteres especiais, coloca tudo em minúsculo, e elimina stopwords.
# O objetivo é limpar o texto para facilitar as próximas análises (intenção, sentimento etc.).
def normalize_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", " ", text)
    try:
        tokens = [t for t in word_tokenize(text, language="portuguese") if t not in stop_words]
    except Exception:
        tokens = text.split()
    return " ".join(tokens).strip()

# 2. Análise de Intenção
# Isso ajuda o chatbot a entender se é uma dúvida de instrução, justificativa, ou identificação.
def analyze_intent(text: str) -> str:
    if any(x in text for x in ["como", "de que forma", "procedimento", "passo", "como faço", "instrução"]):
        return "pedido de instrução"
    if any(x in text for x in ["por que", "motivo", "razão", "porquê"]):
        return "pedido de justificativa"
    if any(x in text for x in ["quem", "órgão", "setor", "quem é", "responsável"]):
        return "pedido de identificação"
    return "outros"

# 3. Análise de Sentimento
# Usa o spaCy para verificar se o texto tem palavras com carga positiva ou negativa.
# Isso permite ajustar o tom das respostas do chatbot (ex: empatia em caso de frustração).
def analyze_sentiment(text: str) -> str:
    try:
        doc = nlp(text)
        positives = sum(1 for tok in doc if tok.lemma_.lower() in {"bom","ótimo","excelente","favorável","positivo","conforme"})
        negatives = sum(1 for tok in doc if tok.lemma_.lower() in {"ruim","erro","problema","falha","negativo","indevido"})
        if positives > negatives:
            return "positivo"
        if negatives > positives:
            return "negativo"
        return "neutro"
    except Exception:
        return "neutro"

# 4. Extração de Entidades
# Identifica nomes de pessoas, órgãos, números de processo, no texto.
def extract_entities(text: str) -> dict:
    doc = nlp(text)
    ents = {}
    for ent in doc.ents:
        ents.setdefault(ent.label_, []).append(ent.text)
    return ents

# 5. Expansão Semântica
# Amplia a pergunta do usuário adicionando termos relacionados semanticamente.
# Isso ajuda o retriever a encontrar documentos relevantes mesmo que usem sinônimos.
def semantic_expansion(text: str) -> str:

    base_terms = ["processo", "documento", "protocolo", "julgamento", "tramitação", "autuação", "andamento", "petição", "decisão", "parecer"]
    try:
        query_emb = embedder.encode(text, convert_to_tensor=True)
        base_embs = embedder.encode(base_terms, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(query_emb, base_embs)[0]
        best_terms = [base_terms[i] for i, s in enumerate(scores) if float(s) > 0.40]
        additions = " ".join([t for t in best_terms if t not in text])
        return (text + " " + additions).strip()
    except Exception:
        return text

"""
Função do pipeline completo
- normaliza o texto
- detecta intenção e sentimento
- extrai entidades
- expande a pergunta semanticamente
"""
def preprocess_input(text: str) -> str:

    normalized = normalize_text(text)
    intent = analyze_intent(normalized)
    sentiment = analyze_sentiment(normalized)
    entities = extract_entities(normalized)
    expanded = semantic_expansion(normalized)

    # Log para debug (aparece no terminal do Streamlit)
    print("[PLN] Intenção:", intent)
    print("[PLN] Sentimento:", sentiment)
    print("[PLN] Entidades:", entities)
    print("[PLN] Pergunta expandida:", expanded)

    return expanded


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Teste das funções

In [ ]:
texto_exemplo = "Quem é responsável por abrir a votação da Sessão de Julgamento?."
print("Texto normalizado:")
print(normalize_text(texto_exemplo))
print("\nIntenção detectada:")
print(analyze_intent(texto_exemplo))
print("\nSentimento detectado:")
print(analyze_sentiment(texto_exemplo))
print("\nEntidades extraídas:")
print(extract_entities(texto_exemplo))
print("\nTexto expandido semanticamente:")
print(semantic_expansion(texto_exemplo))

Texto normalizado:
quem é responsável por abrir a votação da sessão de julgamento

Intenção detectada:
pedido de identificação

Sentimento detectado:
neutro

Entidades extraídas:
{'MISC': ['Sessão de Julgamento?']}

Texto expandido semanticamente:
Quem é responsável por abrir a votação da Sessão de Julgamento?. julgamento


---
## Configuração do Large Language Model (LLM)
Antes de implementar o sistema RAG, definimos e inicializamos o LLM que será responsável por gerar as respostas finais para o usuário.

Neste projeto, utilizamos a Groq Cloud, que se destaca pela sua velocidade de inferência e por ser gratuito.

⚠️ Iremos deixar a nossa chave Groq para realizar os testes !

In [ ]:
import os
from langchain_groq import ChatGroq

id_model = "llama-3.3-70b-versatile"
temperature = 0.7
os.environ["GROQ_API_KEY"] = "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"


##Carregar o LLM

In [ ]:
#Cria a instância do modelo LLM que vai gerar as respostas do chatbot.
def load_llm():
    return ChatGroq(
        model=id_model,
        temperature=temperature,
        api_key=os.getenv("GROQ_API_KEY")
    )

llm = load_llm()

# Teste prático mostrando o funcionamento da llm
llm = load_llm()
resposta = llm.invoke("Explique brevemente o que é o SEI Julgar.")
print(" Resposta do modelo:")
print(resposta.content)


 Resposta do modelo:
O SEI Julgar é um sistema eletrônico do Superior Tribunal de Justiça (STJ) que permite a apresentação de recursos e a realização de julgamentos de forma eletrônica. Ele foi criado para agilizar e modernizar os processos judiciais no STJ, permitindo que os ministros e os advogados realizem suas atividades de forma mais eficiente e transparente.

Com o SEI Julgar, os advogados podem apresentar recursos, petições e outros documentos de forma eletrônica, e os ministros podem analisar e julgar os processos de forma remota. Além disso, o sistema também permite a realização de sessões de julgamento virtuais, o que ajuda a reduzir os custos e o tempo de deslocamento.

O SEI Julgar é uma ferramenta importante para a modernização da justiça brasileira, pois ajuda a tornar os processos judiciais mais ágeis, transparentes e acessíveis.


🚫 O experimento anterior mostrou que, embora o LLM consiga gerar respostas em linguagem natural, ele não possui conhecimento específico sobre o módulo SEI Julgar.
Isso ocorre porque o modelo foi treinado em um conjunto amplo de textos da internet.

---
## Etapa de Indexação

Essa parte do chatbot, é responsável por segmentar, indexar e armazenar os textos extraídos do manual do SEI Julgar e guardá-los em um banco vetorial.

**Visão Geral das Etapas da Indexação**

Basicamente, iniciamos com a extração dos textos do PDF. Em seguida, é realizada a quebra desses textos *(Splitting)*, pois as LLMs possuem um número máximo de caracteres a serem recebidos. Posteriormente, é feito o Embedding, que consiste na transformação dos textos em vetores numéricos. Por fim, ocorre o armazenamento *(Storing)*, onde os dados são persistidos.

| Etapa | Objetivo Principal | Ferramentas Chave |
| :--- | :--- | :--- |
| **1. Extração** | Carregar e extrair o texto bruto do PDF. | `PyPDFLoader` (LangChain) |
| **2. Segmentação (Chunking)** | Dividir o texto em partes menores e gerenciáveis. | `RecursiveCharacterTextSplitter` |
| **3. Geração de Embeddings** | Transformar o texto segmentado em vetores numéricos. | Modelo `BAAI/bge-m3` |
| **4. Armazenamento Vetorial** | Armazenar e persistir os vetores para busca. | `ChromaDB` |
| **5. Mecanismo de Busca** | Realizar a busca dos trechos mais relevantes e diversos. | `Maximal Marginal Relevance (MMR)` |



In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
import os

def config_retriever():


    # Cria o diretório onde o banco vetorial será armazenado
    persist_dir = "./chroma_db"
    os.makedirs(persist_dir, exist_ok=True)

    # 1. Lê o PDF e divide o texto
    pdf_file = "Documentação SEI Julgar - Secretaria.pdf"
    reader = PdfReader(pdf_file)
    texts = [page.extract_text() for page in reader.pages if page.extract_text()]

    # 2.Segmentação (Chunking)
    """
    Chunk_size - Define o tamanho dos chunks
    Chunk_overlap - Define o quanto os chunks se sobrepõem
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.create_documents(texts)

    # 3. Geração de Embeddings
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

    # 4. Cria o banco vetorial com ChromaDB com os embeddings desses chunks
    vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=persist_dir)

    """
    O Maximal Marginal Relevance busca no banco os chunks mais relevantes e diversos para evitar a redundância de informações.
    fetch_k - Define a quantidade de chunks que serão recuperados do banco
    k - Define a quantidade de chunks que serão retornados a llm
    lambda_mult - Controla o peso de Relevância e Diversidade 0.7(70% foco na relevância e 30% na diversidade)
    """
    # 5. Recuperador MMR
    return vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 4, 'lambda_mult': 0.7})





## Teste Prático

In [ ]:

import pandas as pd

retriever = config_retriever()
docs = retriever.invoke("Como criar uma sessão de julgamento ?")

results = []
for d in docs:
    results.append({
        "Trecho Recuperado": d.page_content.replace("\n", " ").strip()[:400] + "..."
    })

pd.DataFrame(results)

/tmp/ipython-input-1816634796.py:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

,Trecho Recuperado
0,2. SECRETARIA DO COLEGIADO 2.1 - ...
1,data e o tipo da sessão (Exemplo: ...
2,Observação: mesmo com a pauta fechad...


---
## Recuperação e Geração

Nessa etapa, são definidos os prompts, as regras de comportamento e o tratamento do histórico, que permitem ao chatbot manter o contexto e conversar de forma contínua com o usuário. Essa fase forma a pipeline RAG, onde o sistema recupera os trechos mais relevantes da base e gera respostas contextuais a partir deles.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def config_rag_chain(llm, retriever):
    """
    Prompt serve para o modelo entender o contexto da conversa e reescrever
    a pergunta do usuário com base no histórico.
    """
    context_prompt = ChatPromptTemplate.from_messages([
        ("system", "Reformule perguntas do usuário com base no histórico, sem respondê-las."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])

    """
    Busca o histórico das conversas para que as respostas sejam contextualizadas
    """
    history_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_prompt
    )

    # Prompt principal do QA
    """
    Esse prompt é responsável por gerar as respostas do chatbot.
    Nele é definido o papel do assistente e a forma como ele deve responder.
    """
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", """
Você é um assistente virtual especializado **exclusivamente** no módulo **SEI Julgar**.
Responda sempre com base na documentação oficial e nunca invente informações.
"""),
        MessagesPlaceholder("chat_history"),
        # Combina a pergunta reformulada (input) com o contexto recuperado
        ("human", "Pergunta: {input}\n\nContexto do manual:\n{context}")
    ])

    #envia a llm o prompt acima mais os documentos retornados pelo retriever
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    """
    Isso forma o pipeline completo RAG
    1. Reformula a pergunta com base no histórico.
    2. Busca documentos relevantes no banco vetorial.
    3. Gera respostas com base nas informações recuperadas.
    """
    return create_retrieval_chain(history_retriever, qa_chain)


## 💬 Teste 1 – Pergunta simples

In [ ]:
rag_chain = config_rag_chain(llm, retriever)
# Simulação de histórico de conversa
chat_history = []

user_input = "O que é o módulo SEI Julgar e qual sua função?"
response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})

print("Pergunta:", user_input)
print("\nResposta:\n", response["answer"])


Pergunta: O que é o módulo SEI Julgar e qual sua função?

Resposta:
 O módulo SEI Julgar é um componente do Sistema Eletrônico de Informações (SEI) que visa a autuação, distribuição, instrução e julgamento de processos administrativos em ambiente digital. Sua função é permitir o acompanhamento eletrônico de todas as etapas processuais, desde a autuação inicial até a decisão final, garantindo maior transparência, eficiência e celeridade nos trâmites internos.

Com o SEI Julgar, é possível criar automaticamente processos para Sessões de Julgamento, eliminar a necessidade de movimentações físicas de processos e permitir que as secretarias dos colegiados gerenciem as sessões de julgamento de forma eletrônica. Além disso, o módulo também permite que as sessões sejam pautadas e fechadas de forma automática, criando um processo para a Sessão de Julgamento sempre que a situação for alterada para "Pauta Fechada".


##💬 Teste 2 – Pergunta contextual (histórico)

In [ ]:
chat_history = [HumanMessage(content="Explique o módulo SEI Julgar."),
                AIMessage(content="O SEI Julgar é um módulo para julgamento de processos administrativos em colegiado.")]

user_input = "E como funciona a parte de pautas?"
response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})

print("Pergunta:", user_input)
print("\nResposta contextualizada:\n", response["answer"])


Pergunta: E como funciona a parte de pautas?

Resposta contextualizada:
 A parte de pautas no módulo SEI Julgar funciona da seguinte maneira:

1. **Criação da Pauta**: Quando a secretaria do colegiado fecha a pauta, o SEI Julgar cria automaticamente um processo para a Sessão de Julgamento.
2. **Pauta Fechada**: Quando a pauta é fechada, o sistema gera automaticamente um processo da sessão que centraliza a documentação do julgamento. Esse processo é composto por vários documentos, incluindo:
 * Pauta de Julgamento: lista os processos a serem julgados na sessão.
 * Ata de Julgamento: documento gerado após a sessão.
3. **Inclusão e Retirada de Processos**: Mesmo com a pauta fechada, o relator ainda pode incluir ou retirar processos em mesa.
4. **Abrir Pauta**: Para iniciar a Sessão de Julgamento, basta abrir a pauta após sua criação. O botão "Abrir a pauta" permite que os gabinetes incluam processos na sessão, tanto na pauta quanto na mesa.
5. **Pauta Aberta**: Enquanto a pauta estiver ab

---

Até aqui percorremos todas as etapas essenciais para a construção do chatbot, integrando técnicas de Processamento de Linguagem Natural (PLN) com o modelo RAG *(Retrieval-Augmented Generation)*.




Agora, usaremos o Streamlit para demonstrar a versão final do chatbot do SEI Julgar, pois é uma ferramenta eficiente e interativa para desenvolver interfaces web em Python e utilizar CSS personalizado.


In [ ]:
%%writefile .env
GROQ_API_KEY=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

Writing .env


Carregue o código chatbot_sei.py

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Arquivo enviado: {filename}")


KeyboardInterrupt: 

Após executar o código a seguir, clique no link em azul https://teams-rough-maximize... que irá aparecer no log da execução.

⚠️ Demora um pouco para base ser criada e o modelo ser carregado, apenas na primeira execução.

In [ ]:
# 1️⃣ Instalar Streamlit e Cloudflared
!pip install streamlit -q
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 2️⃣ Rodar Streamlit em background
!streamlit run /content/chatbot_sei.py &>/content/logs.txt &

# 3️⃣ Expor via Cloudflared
!cloudflared tunnel --url http://localhost:8501



2025-11-11T02:59:01Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-11T02:59:01Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-11T02:59:05Z INF +--------------------------------------------------------------------------------------------+
2025-11-11T02:59:05Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-11T02:59:05Z INF |  https://alternatively-blue-ocean-efficiency.trycloudf

In [ ]:
%%writefile ./chatbot_sei.py
# ************ IMPORTAÇÕES PRINCIPAIS **************************

import streamlit as st
from pathlib import Path
import requests, os, time
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from PyPDF2 import PdfReader


# *********** MÓDULO DE PLN – Pré-processamento e enriquecimento semântico ******************

import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

# Tenta carregar o modelo SpaCy em português (usa versão menor se a maior não estiver disponível)
try:
    nlp = spacy.load("pt_core_news_md")
except Exception:
    try:
        nlp = spacy.load("pt_core_news_sm")
    except Exception:
        st.error("Erro: modelo SpaCy não encontrado. Execute: !python -m spacy download pt_core_news_md")
        st.stop()

# Carrega o modelo de embeddings para cálculo de similaridade semântica
embedder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# Carrega stopwords da língua portuguesa (ou cria lista vazia em caso de falha)
try:
    stop_words = set(stopwords.words("portuguese"))
except Exception:
    stop_words = set()

# Função para normalizar e remover stopwords - Normaliza o texto removendo pontuação, acentuação e stopwords.
def normalize_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", " ", text)
    try:
        tokens = [t for t in word_tokenize(text, language="portuguese") if t not in stop_words]
    except Exception:
        tokens = text.split()
    return " ".join(tokens).strip()

# Função para analizar a intenção do usuário - Identifica a intenção geral do usuário com base em palavras-chave simples.
def analyze_intent(text: str) -> str:
    if any(x in text for x in ["como", "de que forma", "procedimento", "passo", "como faço", "instrução"]):
        return "pedido de instrução"
    if any(x in text for x in ["por que", "motivo", "razão", "porquê"]):
        return "pedido de justificativa"
    if any(x in text for x in ["quem", "órgão", "setor", "quem é", "responsável"]):
        return "pedido de identificação"
    return "outros"

# Função para analisar o sentimento da pergunta - Analisa o sentimento geral da frase com base em palavras positivas e negativas.
def analyze_sentiment(text: str) -> str:
    try:
        doc = nlp(text)
        positives = sum(1 for tok in doc if tok.lemma_.lower() in {"bom", "ótimo", "excelente", "favorável", "positivo", "conforme"})
        negatives = sum(1 for tok in doc if tok.lemma_.lower() in {"ruim", "erro", "problema", "falha", "negativo", "indevido"})
        if positives > negatives:
            return "positivo"
        if negatives > positives:
            return "negativo"
        return "neutro"
    except Exception:
        return "neutro"

# Função para extrair as entidades da pergunta - Extrai entidades nomeadas do texto (pessoas, órgãos, locais, etc).
def extract_entities(text: str) -> dict:
    doc = nlp(text)
    ents = {}
    for ent in doc.ents:
        ents.setdefault(ent.label_, []).append(ent.text)
    return ents

#Função para expandir semânticamente a pergunta - Adiciona termos semanticamente próximos para melhorar a recuperação de contexto.
def semantic_expansion(text: str) -> str:
    base_terms = ["processo", "documento", "protocolo", "julgamento", "tramitação", "autuação", "andamento", "petição", "decisão", "parecer"]
    try:
        query_emb = embedder.encode(text, convert_to_tensor=True)
        base_embs = embedder.encode(base_terms, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(query_emb, base_embs)[0]
        best_terms = [base_terms[i] for i, s in enumerate(scores) if float(s) > 0.40]
        additions = " ".join([t for t in best_terms if t not in text])
        return (text + " " + additions).strip()
    except Exception:
        return text

# Pipeline completo das funções
def preprocess_input(text: str) -> str:
    normalized = normalize_text(text)
    intent = analyze_intent(normalized)
    sentiment = analyze_sentiment(normalized)
    entities = extract_entities(normalized)
    expanded = semantic_expansion(normalized)

    print("[PLN] Intenção:", intent)
    print("[PLN] Sentimento:", sentiment)
    print("[PLN] Entidades:", entities)
    print("[PLN] Pergunta expandida:", expanded)

    return expanded


# ***************CONFIGURAÇÕES DO STREAMLIT E INTERFACE*********************

st.set_page_config(
    page_title="Atendimento SEI Julgar",
    page_icon="⚖️",
    layout="wide",
    initial_sidebar_state="expanded",
)
load_dotenv()

# CSS customizado: define cores, bolhas e estilo visual do chat
st.markdown("""<style> ... </style>""", unsafe_allow_html=True)

# *************SIDEBAR - Configurações do modelo e informações do sistema********************

with st.sidebar:
    st.image("https://www.gov.br/ans/.../sei-logo.png", width=130)
    id_model = st.selectbox("Selecione o modelo:", ["llama-3.3-70b-versatile", "llama-3.1-8b-instant", "mixtral-8x7b-32768"], index=0)
    temperature = st.slider("Temperatura (criatividade):", 0.0, 1.0, 0.7, 0.1)


# ************************** FUNÇÕES DE INDEXAÇÃO ******************************************

@st.cache_resource
def load_llm():
    #Carrega o modelo de linguagem da API Groq
    return ChatGroq(model=id_model, temperature=temperature, api_key=os.getenv("GROQ_API_KEY"))

@st.cache_resource
def config_retriever():

    #Cria ou carrega base vetorial (ChromaDB).

    persist_dir = "./chroma_db"
    os.makedirs(persist_dir, exist_ok=True)

    if os.path.exists(os.path.join(persist_dir, "chroma.sqlite3")):
        vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    else:
      #Carrega o pdf e extrai o texto
        pdf_files = ["Documentação SEI Julgar - Secretaria.pdf"]
        texts = []
        for file in pdf_files:
            if os.path.exists(file):
                reader = PdfReader(file)
                for page in reader.pages:
                    text = page.extract_text()
                    if text:
                        texts.append(text)
         """
        Chunk_size - Define o tamanho dos chunks
        Chunk_overlap - Define o quanto os chunks se sobrepõem
        """
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        chunks = splitter.create_documents(texts)

        # Geração de Embeddings
        embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

        # Cria o banco vetorial com ChromaDB com os embeddings desses chunks
        vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=persist_dir)
        vectorstore.persist()

        """
        O Maximal Marginal Relevance busca no banco os chunks mais relevantes e diversos para evitar a redundância de informações.
        fetch_k - Define a quantidade de chunks que serão recuperados do banco
        k - Define a quantidade de chunks que serão retornados a llm
        lambda_mult - Controla o peso de Relevância e Diversidade 0.7(70% foco na relevância e 30% na diversidade)
        """
    return vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k": 4, "lambda_mult": 0.7})

     # Cria pipeline RAG: reformulação + busca + resposta final.
      def config_rag_chain(llm, retriever):

          """
          Prompt serve para o modelo entender o contexto da conversa e reescrever
          a pergunta do usuário com base no histórico.
          """

        context_prompt = ChatPromptTemplate.from_messages([
            ("system", "Reformule perguntas do usuário sem respondê-las."),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ])
    # Busca o histórico das conversas para que as respostas sejam contextualizadas
    history_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=context_prompt)

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Você é um assistente virtual especializado **exclusivamente** no módulo **SEI Julgar**.
            Forneça respostas **técnicas e objetivas**, baseadas apenas na documentação oficial.

            REGRAS:
            1. Não inventar informações.
            2. Não responder temas fora do SEI Julgar.
            3. Manter tom formal e instrutivo.
            4. Se o tema for irrelevante, diga: "Posso responder apenas sobre o funcionamento do SEI Julgar."
            """),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\nContexto relevante do manual:\n{context}")
    ])

    #envia a llm o prompt acima mais os documentos retornados pelo retriever
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    return create_retrieval_chain(history_retriever, qa_chain)


# ******************* INTERFACE DO CHAT *******************************

def show_message(content, is_user=False):
    """Renderiza mensagens no formato de bolhas."""
    bubble_class = "user-bubble" if is_user else "ai-bubble"
    st.markdown(f"<div class='chat-bubble {bubble_class}'>{content}</div>", unsafe_allow_html=True)

def chat_response(rag_chain, user_input):
    """Processa entrada do usuário e retorna resposta do modelo."""
    st.session_state.chat_history.append(HumanMessage(content=user_input))
    response = rag_chain.invoke({"input": user_input, "chat_history": st.session_state.chat_history})
    answer = response.get("answer", "Erro ao gerar resposta.")
    st.session_state.chat_history.append(AIMessage(content=answer))
    return answer


# ************************* EXECUÇÃO PRINCIPAL ************************************

#Cria o histórico de mensagens
if "chat_history" not in st.session_state:
    st.session_state.chat_history = [AIMessage(content="Olá 👋, sou o assistente do SEI Julgar! Como posso ajudar?")]
# Validação da chave da API
if not os.getenv("GROQ_API_KEY"):
    st.error("❌ GROQ_API_KEY não encontrada.")
    st.stop()
# Inicialização do modelo e da base vetorial
try:
    with st.spinner("Carregando modelo e base vetorial..."):
        llm = load_llm()
        retriever = config_retriever()
        rag_chain = config_rag_chain(llm, retriever)
    st.success("✅ Sistema carregado com sucesso!")

    # Botão de "limpar chat"
    col1, col2 = st.columns([5, 1])
    with col2:
        if st.button("🧹 Limpar chat", key="clear", use_container_width=True):
            st.session_state.chat_history = [AIMessage(content="Chat limpo! 😊 Como posso ajudar agora?")]

    # Exibição do histórico de mensagens
    chat_container = st.container()
    with chat_container:
        for msg in st.session_state.chat_history:
            show_message(msg.content, isinstance(msg, HumanMessage))

    # Captura a entrada do usuário
    user_input = st.chat_input("Digite sua mensagem...")

    # Processamento da resposta
    if user_input:
        with chat_container:
            show_message(user_input, is_user=True)
        placeholder = st.empty()
        placeholder.markdown("<div class='ai-bubble'><em>Digitando...</em></div>", unsafe_allow_html=True)
        with st.spinner("Processando..."):
            answer = chat_response(rag_chain, preprocess_input(user_input))
        placeholder.empty()
        with chat_container:
            show_message(answer, is_user=False)

# Tratamento de erros
except Exception as e:
    st.error(f"Erro ao inicializar o chatbot: {str(e)}")


Writing ./chatbot_sei.py
